<a href="https://colab.research.google.com/github/AtifQureshi110/Fine-Tune/blob/main/sapm_and_ham_locked.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Libraries Installation.

In [ ]:
import warnings
# Ignore all warnings
warnings.filterwarnings("ignore")

In [ ]:
!pip  install transformers==4.22.1 -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.9/4.9 MB 44.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.8/294.8 kB 33.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 106.6 MB/s eta 0:00:00


In [ ]:
import transformers
print(transformers.__version__)

4.22.1


In [ ]:
!pip install -U tensorflow==2.10 -q
!pip install keras -q
!pip install tensorflow_probability==0.12.2 -q

In [ ]:
import tensorflow as tf
import keras
print(tf.__version__)
print(keras.__version__)

2.10.0
2.10.0


### Importing Libraries

In [ ]:
# Load the pre-trained DistilBERT model for sequence classification
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification, TextClassificationPipeline, TFTrainingArguments
import tensorflow as tf

In [ ]:
import pandas as pd

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load tokenizer, model, and pipeline

In [ ]:
# Define pretrained tokenizer and model
model_id = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(model_id)
model = TFDistilBertForSequenceClassification.from_pretrained(model_id, num_labels=2)
#pipeline = TextClassificationPipeline(model=model, tokenizer=tokenizer, device=0)  # Specify device if needed

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_19', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

### Data

In [ ]:
# data from https://www.kaggle.com/datasets/balaka18/email-spam-classification-dataset-csv

file_path="/content/drive/MyDrive/BERT/spam_ham_Bert_binary/spam.csv"

df = pd.read_csv(file_path,encoding = "ISO-8859-1")

In [ ]:
df.drop(['Unnamed: 2',	'Unnamed: 3',	'Unnamed: 4'], axis=1, inplace=True)

In [ ]:
df.rename(columns={'v1':'type','v2':'text'},inplace=True)

In [ ]:
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
df['type_encoded'] = label_encoder.fit_transform(df['type'])

In [ ]:
df["text_word_count"] = df['text'].apply(lambda x: len(x.split()))

In [ ]:
df

,type,text,type_encoded,text_word_count
0,ham,"Go until jurong point, crazy.. Available only ...",0,20
1,ham,Ok lar... Joking wif u oni...,0,6
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,1,28
3,ham,U dun say so early hor... U c already then say...,0,11
4,ham,"Nah I don't think he goes to usf, he lives aro...",0,13
...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,1,30
5568,ham,Will Ì_ b going to esplanade fr home?,0,8
5569,ham,"Pity, * was in mood for that. So...any other s...",0,10
5570,ham,The guy did some bitching but I acted like i'd...,0,26


### Selecting Target column

In [ ]:
X = df['text'].to_list()
y = df['type_encoded'].to_list()

In [ ]:
#Train Test SPlit
X_train, y_train, X_test, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0 )

In [ ]:
print(f"X_train :{len(X_train)}\ny_train: {len(y_train)}\nX_test:{len(X_test)}\ny_test: {len(y_test)}")

X_train :4457
y_train: 1115
X_test:4457
y_test: 1115


### Preprocess your text data

In [ ]:
# ----- 1. Preprocess data -----#
train_encodings = tokenizer(X_train, truncation=True, padding=True, return_tensors='tf')
test_encodings = tokenizer(y_train, truncation=True, padding=True, return_tensors='tf')

In [ ]:
train_labels = np.array(X_test)
test_labels = np.array(y_test)

### Create TensorFlow datasets

In [ ]:
# Create TensorFlow datasets
train_dataset = tf.data.Dataset.from_tensor_slices((dict(train_encodings), train_labels))
test_dataset = tf.data.Dataset.from_tensor_slices((dict(test_encodings), test_labels))

### Define and configure training_args

In [ ]:
# ----- 2. Fine-tune pretrained model -----#
training_args = TFTrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    evaluation_strategy="steps",
    eval_steps=500,
    save_steps=500,
)

In [ ]:
trainer_model = TFDistilBertForSequenceClassification.from_pretrained(model_id, num_labels=2)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_39', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
def build_model():
    model = TFDistilBertForSequenceClassification.from_pretrained(model_id, num_labels=2)

    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=training_args.learning_rate),

                  loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),

                  metrics=['accuracy'])

    return model

In [ ]:
# Create your model within the distribution strategy scope
with training_args.strategy.scope():
    trainer_model = build_model()

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_projector', 'activation_13', 'vocab_transform', 'vocab_layer_norm']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['dropout_59', 'pre_classifier', 'classifier']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
# fine tuning
trainer_model.fit(train_dataset.batch(training_args.per_device_train_batch_size), epochs=training_args.num_train_epochs)

Epoch 1/3
558/558 [==============================] - 134s 213ms/step - loss: 0.0733 - accuracy: 0.9814
Epoch 2/3
558/558 [==============================] - 119s 214ms/step - loss: 0.0266 - accuracy: 0.9933
Epoch 3/3
558/558 [==============================] - 119s 213ms/step - loss: 0.0169 - accuracy: 0.9960


In [ ]:
results = trainer_model.evaluate(test_dataset.batch(training_args.per_device_eval_batch_size))
print("Evaluation results:", results)

18/18 [==============================] - 9s 372ms/step - loss: 0.0281 - accuracy: 0.9892
Evaluation results: [0.028064459562301636, 0.9892376661300659]


In [ ]:
# Predict labels for the test dataset
predictions = trainer_model.predict(test_dataset.batch(training_args.per_device_eval_batch_size))
predictions

18/18 [==============================] - 8s 370ms/step


TFSequenceClassifierOutput(loss=None, logits=array([[ 4.5625515, -4.238049 ],
       [ 4.538792 , -4.227904 ],
       [ 3.999307 , -3.713762 ],
       ...,
       [ 4.540735 , -4.2453303],
       [ 4.5586953, -4.243898 ],
       [ 4.305965 , -3.972886 ]], dtype=float32), hidden_states=None, attentions=None)

In [ ]:
predictions

TFSequenceClassifierOutput(loss=None, logits=array([[ 4.5625515, -4.238049 ],
       [ 4.538792 , -4.227904 ],
       [ 3.999307 , -3.713762 ],
       ...,
       [ 4.540735 , -4.2453303],
       [ 4.5586953, -4.243898 ],
       [ 4.305965 , -3.972886 ]], dtype=float32), hidden_states=None, attentions=None)

In [ ]:
# # Define a threshold value for converting scores to labels
# threshold = 0.5  # Adjust this threshold as needed

# # Assuming df is your DataFrame
# df['predicted_label'] = np.where(predictions.logits > threshold, 1, 0)

# # Display the DataFrame with predicted labels
# print(df)

In [ ]:
# from transformers import TFTrainer, TFTrainingArguments
# import tensorflow as tf

# # Get the predicted labels from the predictions
# #predicted_labels = predictions.predictions.argmax(axis=1).numpy()
# predicted_labels = tf.argmax(predictions.logits, axis=1).numpy()

# # Get the actual labels from the test dataset
# actual_labels = [label.numpy() for _, label in test_dataset]

# # Compare the predicted labels with the actual labels
# for i in range(len(predicted_labels)):
#     print(f"Example {i + 1} - Actual: {actual_labels[i]}, Predicted: {predicted_labels[i]}")

In [ ]:
# # Fit the model to the training dataset
# #trainer_model.fit(train_dataset.batch(training_args.per_device_train_batch_size), epochs=training_args.num_train_epochs)
# model.fit(train_dataset.batch(training_args.per_device_train_batch_size), epochs=training_args.num_train_epochs)
# # Predict labels for the test dataset
# #predictions = trainer_model.predict(test_dataset.batch(training_args.per_device_eval_batch_size))
# predictions = model.predict(test_dataset.batch(training_args.per_device_eval_batch_size))

# # Get the predicted labels and logits from the predictions
# predicted_labels = tf.argmax(predictions.predictions, axis=1).numpy()
# predicted_logits = predictions.predictions.numpy()

# # Get the actual labels from the test dataset
# actual_labels = [label.numpy() for _, label in test_dataset]


### Saving & Loading the model

In [ ]:
#Saving & Loading the model
save_directory = "/saved_models"
model.save_pretrained(save_directory)

In [ ]:
tokenizer.save_pretrained(save_directory)

('/saved_models/tokenizer_config.json',
 '/saved_models/special_tokens_map.json',
 '/saved_models/vocab.txt',
 '/saved_models/added_tokens.json')

### Loading Pre-Trained Model

In [ ]:
#Loading Pre-Trained Model
tokenizer_fine_tuned = DistilBertTokenizer.from_pretrained(save_directory)
model_fine_tuned = TFDistilBertForSequenceClassification.from_pretrained(save_directory)

Some layers from the model checkpoint at /saved_models were not used when initializing TFDistilBertForSequenceClassification: ['dropout_19']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at /saved_models and are newly initialized: ['dropout_79']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
y_train

['Aight should I just plan to come up later tonight?',
 'Was the farm open?',
 'I sent my scores to sophas and i had to do secondary application for a few schools. I think if you are thinking of applying, do a research on cost also. Contact joke ogunrinde, her school is one me the less expensive ones',
 'Was gr8 to see that message. So when r u leaving? Congrats dear. What school and wat r ur plans.',
 "In that case I guess I'll see you at campus lodge",
 "Nothing will ever be easy. But don't be looking for a reason not to take a risk on life and love",
 'If you were/are free i can give. Otherwise nalla adi entey nattil kittum',
 "Hey i will be late... i'm at amk. Need to drink tea or coffee",
 'Hey are we going for the lo lesson or gym? ',
 '85233 FREE>Ringtone!Reply REAL',
 '\\The world suffers a lot... Not because of the violence of bad people. But because of the silence of good people!\\"',
 'Hey!!! I almost forgot ... Happy B-day babe ! I love ya!!',
 'I wish u were here. I feel s

In [ ]:
y_trai = y_train[2]
y_trai

'I sent my scores to sophas and i had to do secondary application for a few schools. I think if you are thinking of applying, do a research on cost also. Contact joke ogunrinde, her school is one me the less expensive ones'

In [ ]:
predict_input = tokenizer_fine_tuned.encode(
    y_trai,
    truncation = True,
    padding = True,
    return_tensors = 'tf'
)

output = model_fine_tuned(predict_input)[0]

prediction_value = tf.argmax(output, axis = 1).numpy()[0]

prediction_value

1